In [ ]:
# Original example: https://github.com/microsoft/autogen/blob/main/notebook/agentchat_function_call.ipynb

In [1]:
from autogen import config_list_from_json, UserProxyAgent, AssistantAgent
from datetime import datetime
import os, requests, json

In [2]:
# Note; order of model matters
config_list = config_list_from_json(
    env_or_file = "../OAI_CONFIG_LIST",
    filter_dict = {"model": ["gpt4-autogen", "gpt-4", "azuregpt", "gpt-3.5-turbo"]}
)

In [3]:
# Define functions
def address_search(query: str) -> str | None:
    params = {"q": query}
    resp = requests.get("https://geocode.maps.co/search", params=params)
    if resp.status_code == 200:
        return resp.text

In [4]:
llm_config = {
    "functions": [
        {
            "name": "address-search",
            "description": "Will give you a list of geocodes matching an address query. Use this to get the geocoordinates of any address or place.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "An address, city, place or location",
                    }
                },
                "required": ["query"],
            },
        }
    ],
    "config_list": config_list,
    "timeout": 120,
}

In [5]:
user_proxy = UserProxyAgent(
    name="user_proxy",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    human_input_mode="ALWAYS",
    max_consecutive_auto_reply=10
    )

In [6]:
# register the functions
user_proxy.register_function(
    function_map={
        "address-search": address_search,
    }
)

In [7]:
chatbot = AssistantAgent(
    name="chatbot",
    #system_message="For research tasks, only use the functions you have been provided with. Reply TERMINATE when the task is done.",
    llm_config=llm_config,
)

In [8]:
# start the conversation
user_proxy.initiate_chat(
    chatbot,
    message="What is the distance between the Munch museum in Oslo and Giralda, the cathedral in Seville, Spain in metres?",
    clear_history=False
)

user_proxy (to chatbot):

What is the distance between the Munch museum in Oslo and Giralda, the cathedral in Seville, Spain in metres?

--------------------------------------------------------------------------------
chatbot (to user_proxy):

***** Suggested function Call: address-search *****
Arguments: 
{"query":"Munch museum, Oslo"}
***************************************************

--------------------------------------------------------------------------------

>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING FUNCTION address-search...
user_proxy (to chatbot):

***** Response from calling function "address-search" *****
[{"place_id":217363540,"licence":"Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright","powered_by":"Map Maker: https://maps.co","osm_type":"way","osm_id":545260792,"boundingbox":["59.9052701","59.9061782","10.7545583","10.7559512"],"lat":"59.905623899999995","lon":"10.755155448278213","display_name":"MUNCH, 

execute_code was called without specifying a value for use_docker. Since the python docker package is not available, code will be run natively. Note: this fallback behavior is subject to change


user_proxy (to chatbot):

exitcode: 0 (execution succeeded)
Code output: 
The distance between the Munch Museum in Oslo and Giralda in Seville is: 2771684.29 meters


--------------------------------------------------------------------------------
chatbot (to user_proxy):

The distance between the Munch Museum in Oslo and the Giralda tower in Seville is approximately 2,771,684.29 meters (or about 2,771 kilometers).

If you need any further assistance, feel free to ask. Otherwise, is there anything else I can help you with today?

TERMINATE

--------------------------------------------------------------------------------
